Converting the price and the units into € / 100 gramms

In [1]:
import pandas as pd
import numpy as np

feb = pd.read_csv('/Users/david/Documents/GitHub/Final_project_DGU_LDU_COFFEE/2_data_cleaning&analysis/1_cleaning/cleaned_coffee_df_Temp.csv')

price = feb['est_price']
price = pd.DataFrame(price)
price['est_price'] = feb['est_price']
price['blind_assessment'] = feb['blind_assessment'] 

We move in a new column everything after the '/' in a new column

In [2]:
split_values = price['est_price'].str.split('/')
# Create a new column 'unit' with the second part of the split values
price['after_slash'] = split_values.str[1]

We move in a new column everything before the '/' in a new column

In [3]:
price['before_slash'] = price['est_price'].apply(lambda x: x.split('/')[0])

We move in a new column everything before the '/' in a new column

In [4]:
price['before_DOL'] = price['est_price'].apply(lambda x: x.split('$')[0])

In [5]:
#We remove the currency symbol
split_values = price['before_slash'].str.split('$')

# Create a new column 'price_new' with the second part of the split values
price['rem_D'] = split_values.str[1]

In [6]:
replace_dict = {'NT': '', '.': ','}
price['rem_D'] = price['rem_D'].replace(replace_dict)

In [7]:
price.loc[1893, 'rem_D']

'NT 700'

In [8]:
#One string is still reminding :
price.loc[1893, 'rem_D'] = price.loc[1893, 'rem_D'].replace('NT', '', 1893)

In [9]:
price['rem_D'].dtypes

dtype('O')

In [10]:
price['rem_D'] = price['rem_D'].str.strip()

In [11]:
# We identify the empty value which provide a ""
empty_rows = price[price['rem_D'] == '']
empty_rows

,est_price,blind_assessment,after_slash,before_slash,before_DOL,rem_D
170,$NT$520/200g,"Resonantly floral, fruit-driven. Narcissus, po...",200g,$NT$520,,
171,$NT$1000/200g,"Sweet-tart, gently herbaceous. Dried apricot, ...",200g,$NT$1000,,


In [12]:
price.loc[171, 'rem_D'] = 1000
price.loc[170, 'rem_D'] = 520

In [13]:
price.dropna(subset=['rem_D'], inplace=True)

In [14]:
price['rem_D'] = price['rem_D'].str.replace(',', '')

In [15]:
price.loc[217, 'rem_D'] = price.loc[217, 'rem_D'].replace(',', '', 217)
price.loc[219, 'rem_D'] = price.loc[219, 'rem_D'].replace(',', '', 219)

In [16]:
price.loc[217, 'rem_D']

'1500'

In [17]:
price.loc[219, 'rem_D']

'1500'

We will move the rest of the String from 'rem_D' to 'before_DOL'

In [18]:
price['rem_D'] = pd.to_numeric(price['rem_D'], errors='coerce')

In [19]:
price['rem_D'] = price['rem_D'].astype(float)

In [20]:
# For CAD Canadien Dollar
for index, row in price.iterrows():
    if isinstance(row['rem_D'], float):
        # Handle NaN or other float values as needed
        continue

    if 'CAD' in row['rem_D']:
        row['before_DOL'] += 'CAD '
        row['rem_D'] = row['rem_D'].replace('CAD', '').strip()
# Same for Laotian Kip Currency

for index, row in price.iterrows():
    if isinstance(row['rem_D'], float):
        # Handle NaN or other float values as needed
        continue

    if 'LAK' in row['rem_D']:
        row['before_DOL'] += 'LAK '
        row['rem_D'] = row['rem_D'].replace('LAK', '').strip()
        
# Same for New Taiwan dollar Currency

for index, row in price.iterrows():
    if isinstance(row['rem_D'], float):
        # Handle NaN or other float values as needed
        continue

    if 'NTD' in row['rem_D']:
        row['before_DOL'] += 'NTD '
        row['rem_D'] = row['rem_D'].replace('NTD', '').strip()

In [21]:
#To simplify the database we drop the rest of the record which lack of informations
missing_count = price['rem_D'].isna().sum()
missing_count

7

In [22]:
price.dropna(subset=['rem_D'], inplace=True)

**Currency convertion into EURO**

In [23]:
# First, let´s check the uniques currency
price['before_DOL'].unique()

array(['', 'NT ', 'NT', 'CAD ', 'HKD ', 'HK ', 'US ', 'USD ', 'IDR ',
       'NTD ', 'AED ', 'AUD ', 'KRW ', 'THB ', 'RMB ', 'CNY ', 'TWD ',
       'KRW', 'NTD', 'Price: '], dtype=object)

In [24]:
#We create a column named 'Euro'
price['Euro'] = pd.Series('', index=price.index)

In [25]:
# converting the US Dollar in price['Euro']
# US Dollar to EURO (0,95): 'US ', 'USD ', 'Price'
price.loc[price['before_DOL'].str.contains('US\s*|USD\s*|Price\s*', regex=True), 'Euro'] = price['rem_D'] * 0.95
price['Euro'] = price.apply(lambda x: x['rem_D']*0.95 if pd.isna(x['before_DOL']) or x['before_DOL'] == '' else x['Euro'], axis=1)

In [26]:
# New Taiwan dollar to Euro (0,031 Euro) :'NT ', 'NT', 'NTD ', 'NTD', 'TWD'
price.loc[price['before_DOL'].str.contains('NT\s*|NTD\s*|TWD\s*', regex=True), 'Euro'] = price['rem_D'] * 0.031

# CAD dollar to Euro (0.69): 'CAD '
price.loc[price['before_DOL'].str.contains('CAD ', regex=True), 'Euro'] = price['rem_D'] * 0.69

# Hong Kong Dollar to Euro (0,12): 'HKD ','HK '
price.loc[price['before_DOL'].str.contains('HKD\s*|HK\s*', regex=True), 'Euro'] = price['rem_D'] * 0.12

# Indonesian Rupiah to Euro (0,000061):'IDR ',
price.loc[price['before_DOL'].str.contains('IDR\s*', regex=True), 'Euro'] = price['rem_D'] * 0.000061

# United Arab Emirates Dirham to Euro (0,26):'AED ',
price.loc[price['before_DOL'].str.contains('AED\s*', regex=True), 'Euro'] = price['rem_D'] * 0.26

# Australian Dollar to Euro (0,63):'AUD ',
price.loc[price['before_DOL'].str.contains('AUD\s*', regex=True), 'Euro'] = price['rem_D'] * 0.63

# South Korean won to Euro (0,00072):'KRW ',
price.loc[price['before_DOL'].str.contains('KRW\s*', regex=True), 'Euro'] = price['rem_D'] * 0.00072

# Thai Baht to Euro (0,027):'THB ',
price.loc[price['before_DOL'].str.contains('THB\s*', regex=True), 'Euro'] = price['rem_D'] * 0.027

# Chinese Yuan to Euro (0,14):'RMB ', 'CNY'
price.loc[price['before_DOL'].str.contains('RMB\s*|CNY\s*', regex=True), 'Euro'] = price['rem_D'] * 0.14

**Weight units convertion into 100 gramms**

In [27]:
#We create a column named '100_gr_EUR'
price['100_gr_EUR'] = pd.Series('', index=price.index)

In [28]:
#We look for missing value and drop them
missings_count = price['after_slash'].isna().sum()
missings_count

4

In [29]:
price.dropna(subset=['after_slash'], inplace=True)

In [47]:
# 12 ounces : '12 ounce', 12 ounces, 12-ounce bottle, 12 oz, 12 onces, 340, 12 ouncues
price.loc[price['after_slash'].str.contains('12 ounces\s*| 12 oz\s*|12 oz\s*|12-ounce\s*|12 ounce\s*|12 onces\s*|12 ouncues\s*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.29

# , 8 ounces,' 8 ounces',' 8 oz.', 8 oz., 227 grams , 227g, ' 227 grams'
price.loc[price['after_slash'].str.contains('8 ounces\s*|8-ounce\s*|8 ounces\s*|227 grams\s*|227g\s*|227 grams\s*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.29

# 4 ounces , 4 ounces, 120 grams, 115 grams
price.loc[price['after_slash'].str.contains('(?=.*4 ounces\s)*|(?=.* 4 ounces\s)*|(?=.*120 grams\s)*|(?=.*115 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.88

# 10 ounces, 285 grams
price.loc[price['after_slash'].str.contains('(?=.*10 ounces\s)*|(?=.*285 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.35

# 6 ounces, 
price.loc[price['after_slash'].str.contains('(?=.*6 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.59

# 200 grams, 200g, 200 g., 7 ounces, 7 oz.,
price.loc[price['after_slash'].str.contains('(?=.*200 grams\s)*|(?=.* 200 g.\s)*|(?=.*200g\s)*|(?=.*7 ounces\s)*|(?=.* 7 oz.\s)*|(?=.*7 oz.\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.50

# 60 grams, 
price.loc[price['after_slash'].str.contains('(?=.*60 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 1.67

# 16 ounces, ' 16 ounces', ' 16 ounces.', 16ounces,' 16 ounces.',16 oz, 454 grams, 453 grams, 460 g,
price.loc[price['after_slash'].str.contains('(?=.*16 ounces\s)*|(?=.* 16 ounces\s)*|(?=.*453 grams\s)*|(?=.*16ounces\s)*|(?=.*16 ounces.\s)*|(?=.*16 oz\s)*|(?=.*454 grams.\s)*|(?=.*460 grams.\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.22

# 230 grams, 230g
price.loc[price['after_slash'].str.contains('(?=.*230 grams\s)*|(?=.*230g\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.44

# 150 grams, 150-gram, 5.5 ounces
price.loc[price['after_slash'].str.contains('(?=.*150 grams\s)*|(?=.*150-gram\s)*|(?=.*5.5 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.67

# 225 grams, 225 gram,' 225 grams',
price.loc[price['after_slash'].str.contains('(?=.*225 gram\s)*|(?=.* 225 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.44

# 14 ounces, 400 grams
price.loc[price['after_slash'].str.contains('(?=.*14 ounces\s)*|(?=.*400 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.25

# 240 grams
price.loc[price['after_slash'].str.contains('(?=.*240 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.42

# 250 g., 250 gram, ' 250 g.',250 g., 8.8 oz, 8.8 ounces
price.loc[price['after_slash'].str.contains('(?=.*250 g\s)*|(?=.*250 gram\s)*|(?=.* 250 g\s)*|(?=.*8.8\s)*|(?=.*8.8 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.40

# 100 grams, 3.5 ounces, 105 grams
price.loc[price['after_slash'].str.contains('(?=.*100 grams\s)*|(?=.*3.5 ounces\s)*|(?=.*105 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 1

# 50 gram, 50 g. 
price.loc[price['after_slash'].str.contains('(?=.*50 gram\s)*|(?=.*50 g. \s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 2.00

# 20 grams, 21 grams
price.loc[price['after_slash'].str.contains('(?=.*20 grams\s)*|(?=.*21 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 5.00

# 175 grams
price.loc[price['after_slash'].str.contains('(?=.*175 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] *0.57

# 10.5 ounces, 300 grams
price.loc[price['after_slash'].str.contains('(?=.*10.5 ounces\s)*|(?=.*300 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] *0.33

# 5 ounces
price.loc[price['after_slash'].str.contains('(?=.*5 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] *0.71

# 220 grams, 
price.loc[price['after_slash'].str.contains('(?=.*220 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] *0.45

# 5.5 ounces
price.loc[price['after_slash'].str.contains('(?=.*5.5 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] *0.64

# 500 grams, 500 grams, 500 g. 
price.loc[price['after_slash'].str.contains('(?=.*500 grams\s)*|(?=.*500 grams\s)*|(?=.*500 g.\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.20

# 12.64 ounces, 
price.loc[price['after_slash'].str.contains('(?=.*12.64 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.28

# 11 ounces
price.loc[price['after_slash'].str.contains('(?=.*11 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] *0.32

# 20 ounces, 
price.loc[price['after_slash'].str.contains('(?=.*20 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.18

# 32 ounces,
price.loc[price['after_slash'].str.contains('(?=.*32 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.11

# 22 ounces,
price.loc[price['after_slash'].str.contains('(?=.*22 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.16

# 2 ounces,
price.loc[price['after_slash'].str.contains('(?=.*2 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 1.79

# 30 ounces, 2 pounds, 
price.loc[price['after_slash'].str.contains('(?=.*30 ounces\s)*|(?=.*2 pounds\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.12

# 70 grams
price.loc[price['after_slash'].str.contains('(?=.* 70 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 1.43

# 24 ounces
price.loc[price['after_slash'].str.contains('(?=.*24 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.15

# 36 ounces
price.loc[price['after_slash'].str.contains('(?=.*36 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.10

# 11.3 ounces, 320 grams
price.loc[price['after_slash'].str.contains('(?=.*11.3 ounces\s)*|(?=.*320 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.31

# 28 ounces
price.loc[price['after_slash'].str.contains('(?=.*28 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.13

# 10.3 ounces'
price.loc[price['after_slash'].str.contains('(?=.*10.3 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.34

# 13 ounces
price.loc[price['after_slash'].str.contains('(?=.*13 ounces\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.27

# 165 grams
price.loc[price['after_slash'].str.contains('(?=.*165 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.61
# 6 pounds
price.loc[price['after_slash'].str.contains('(?=.*6 pounds\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.04

# 1 kilogram, 1 kg
price.loc[price['after_slash'].str.contains('(?=.*1 kg\s)*|(?=.*320 grams\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 0.10

# 0.94 oz, 0.93 oz
price.loc[price['after_slash'].str.contains('(?=.*0.94\s)*|(?=.*0.93\s)*', regex=True), '100_gr_EUR'] = price['Euro'] * 3.85

In [41]:
## First, let´s check the uniques units
price['after_slash'].unique()

array(['12 ounces', '8 ounces', '4 ounces', '227 grams', '10 ounces',
       '6 ounces', '200 grams', '60 grams', '16 ounces', '230 grams',
       '150 grams', '225 grams', '14 ounces', '240 grams', '250 grams',
       '100 grams', '50 grams', '230g', '200g', '454 grams', ' 12 ounces',
       '20 grams', ' 8 ounces', '175 grams', '285 grams', '10.5 ounces',
       '70 grams; $200.00', '120 grams', '455 grams', '113 grams',
       '400 grams', '227g', '115 grams', '24 ounces', '275 grams',
       '300 grams', '7 ounces', ' 227 grams', '5 ounces', '220 grams',
       '105 grams', '5.5 ounces', '10 capsules', '12.64 ounces',
       'six 8-ounce cans', '375 ml. flask', '150-gram tin',
       'twelve 6-ounce cans', 'six 12-ounce cans', '12-ounce bottle',
       'four 8-ounce cans', '280 ml can', '200-ml bottle',
       '160-ml bottle', '453 grams', '500 grams', '460 grams',
       '11 ounces', 'seven single-serve pouches', '20 ounces (2 types)',
       '32 ounces', '22 ounces', ' 7 ounces',

In [53]:
price = price.drop(price[price['after_slash'] == 'six 8-ounce cans' or '375 ml. flask' or 'twelve 6-ounce cans'or 'six 12-ounce cans' or'four 8-ounce cans'or '280 ml can' or'six 12-ounce cans'or '200-ml bottle' or '160-ml bottle' or'seven single-serve pouches' or 'sachet (plus one donated)'or 'five 5-gram single-serve packets'or  'eight 3.3 gram packets'or 'eight 5-gram tubes'or 'six 5-gram single-serve packets'or  'six 5-gram packets'or 'thirty 1.6-gram single-serve packets'or '12 capsules' or  '24 K-Cups' or'16 K-cups' or '18 K-Cups' or '24 K-cups' or '16 K-Cups'or '32 K-Cups'or '18 capsules'or '18 K-Cups'or '24 K-cups'or '10 single-serve capsules'or'12 K-Cups' or'14 single-serve capsules' or '12 single-serve pods'or'12 single-serve cups'or '24 RealCup capsules'or'48 RealCup capsules' or '12 T Discs' or '22 K-Cups'or '108 pods' ].index)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [55]:
price = price.drop(price[(price['after_slash'] == 'six 8-ounce cans') | 
                          (price['after_slash'] == '375 ml. flask') | 
                          (price['after_slash'] == 'twelve 6-ounce cans') | 
                          (price['after_slash'] == 'six 12-ounce cans') | 
                          (price['after_slash'] == 'four 8-ounce cans') | 
                          (price['after_slash'] == '280 ml can') | 
                          (price['after_slash'] == 'six 12-ounce cans') | 
                          (price['after_slash'] == '200-ml bottle') | 
                          (price['after_slash'] == '160-ml bottle') | 
                          (price['after_slash'] == 'seven single-serve pouches') | 
                          (price['after_slash'] == 'sachet (plus one donated)') | 
                          (price['after_slash'] == 'five 5-gram single-serve packets') | 
                          (price['after_slash'] == 'eight 3.3 gram packets') | 
                          (price['after_slash'] == 'eight 5-gram tubes') | 
                          (price['after_slash'] == 'six 5-gram single-serve packets') | 
                          (price['after_slash'] == 'six 5-gram packets') | 
                          (price['after_slash'] == 'thirty 1.6-gram single-serve packets') | 
                          (price['after_slash'] == '12 capsules') | 
                          (price['after_slash'] == '24 K-Cups') | 
                          (price['after_slash'] == '16 K-cups') | 
                          (price['after_slash'] == '18 K-Cups') | 
                          (price['after_slash'] == '24 K-cups') | 
                          (price['after_slash'] == '16 K-Cups') | 
                          (price['after_slash'] == '32 K-Cups') | 
                          (price['after_slash'] == '18 capsules') | 
                          (price['after_slash'] == '18 K-Cups') | 
                          (price['after_slash'] == '24 K-cups') | 
                          (price['after_slash'] == '10 single-serve capsules') | 
                          (price['after_slash'] == '12 K-Cups') | 
                          (price['after_slash'] == '14 single-serve capsules') | 
                          (price['after_slash'] == '12 single-serve pods') | 
                          (price['after_slash'] == '12 single-serve cups') | 
                          (price['after_slash'] == '24 RealCup capsules') | 
                          (price['after_slash'] == '48 RealCup capsules') | 
                          (price['after_slash'] == '12 T Discs') | 
                          (price['after_slash'] == '22 K-Cups') | 
                          (price['after_slash'] == '108 pods')].index)


In [57]:
price['100_gr_EUR'] = price['100_gr_EUR'].astype(float)

In [58]:
price['100_gr_EUR'] = price['100_gr_EUR'].round(2)

In [60]:
count = price['after_slash'].value_counts()['48 RealCup capsules']
count

KeyError: '48 RealCup capsules'

In [61]:
price.to_csv('Price_weight_cleaned.csv')